In [1]:
#!pip install EnsemblePursuit
from EnsemblePursuit.EnsemblePursuit import EnsemblePursuit
import numpy as np

# Introduction

In this notebook we work with a large-scale calcium imaging recording from the primary visual cortex V1 of the mouse available publicly https://figshare.com/articles/Recording_of_19_000_neurons_across_mouse_visual_cortex_during_sparse_noise_stimuli/9505250 

We apply a sparse matrix factorization algorithm called EnsemblePursuit to extract ensembles of correlated cells from this dataset. The algorithm is available here https://github.com/MouseLand/EnsemblePursuit It can be pip installed ('pip install EnsemblePursuit').

The following notebook explored the relationship between the spatial organization of cells in the ensemble and their relationship to stimuli in receptive fields and behavioral data in the form of motion energy of the mouse's face extracted via the SVD algorithm, https://github.com/MouseLand/EnsemblePursuit/blob/master/Notebooks/BehaviorStimulusCorrsWithSpatialSpreadOfEnsemble.ipynb Specifically, we use ridge regression to map from stimulus and behavioral components to the average activity of the cells in ensemble and quantify how well the learned models can predict the activity of the ensemble on test data (quantified through correlation of the predicted timeseries with the actual timeseries). 

We found that ensembles that exhibit linear receptive fields are localized in space while ensembles that are best explained by behavior display higher spatial variance. 

In this notebook we take inspiration from a paper that found that the synapses between neurons with similar receptive fields are very strong, while synapses that are formed between cells that do not show correlation in their responses are weak. 

We create an artificial neuron that receives strong inputs from a "sensory" ensemble (ensemble with a receptive field) and weak inputs from a spatially spread out "behavioral ensemble" (ensemble taht is well predicted by behavioral motion SVD's). We endow the neuron with the BCM plasticity rule and study the evolution of synaptic weights and the change in recepti

# Extracting Ensembles of Correlated Cells

In [2]:

data_path='/media/maria/DATA1/Documents/data_for_suite2p/TX39/'
dt=1
spks= np.load(data_path+'spks.npy')
print('Shape of the data matrix, neurons by timepoints:',spks.shape)
iframe = np.load(data_path+'iframe.npy') # iframe[n] is the microscope frame for the image frame n
ivalid = iframe+dt<spks.shape[-1] # remove timepoints outside the valid time range
iframe = iframe[ivalid]
S = spks[:, iframe+dt]
print(S.shape)
#Uncomment to compute U and V
#ep=EnsemblePursuit(n_components=50,lam=0.01,n_kmeans=50)
#model=ep.fit(S.T[:10000,:])
#V=model.components_
#U=model.weights
#np.save('U.npy',U)

Shape of the data matrix, neurons by timepoints: (18795, 30766)
(18795, 30560)
obtained 50 PCs in 7.1065 seconds
initialized 50 clusters with k-means in 19.2944 seconds
ensemble 0, time 13.25, nr neurons 6337, EV 0.0086
ensemble 25, time 65.89, nr neurons 268, EV 0.0333
ensemble 49, time 95.60, nr neurons 130, EV 0.0420
average sparsity is 0.0274


In [3]:
del spks

In [7]:
#Select stim ensemble, make weights for stim ens
print(print(U.shape))
stim_ens_inds=np.nonzero(U[:,13])[0]
print(stim_ens_inds.shape)
stim_k=7.5
stim_theta=1.0
stim_weights=np.random.gamma(shape=stim_k,scale=stim_theta,size=(stim_ens_inds.shape[0],))

(18795, 50)
None
(642,)


In [9]:
#Select beh ensemble, make weights for stim_ens
beh_ens_inds=np.nonzero(U[:,8])[0]
print(beh_ens_inds.shape)
beh_k=1.0
beh_theta=2.0
beh_weights=np.random.gamma(shape=beh_k,scale=beh_theta,size=(beh_ens_inds.shape[0],))

(463,)
